# Relay Example Notebook

This notebook demonstrates how to use Relay to:
1. Submit batch jobs
2. Track job progress
3. Retrieve results from old jobs

The key feature of Relay is that all jobs and results are stored in a workspace directory, so you can access them across sessions.

## Setup

First, let's import the necessary modules and set up the workspace.

In [ ]:
import os
import time
from relay import RelayClient, BatchRequest

# Check if API key is set
if not os.getenv("OPENAI_API_KEY"):
    print("⚠️  Warning: OPENAI_API_KEY not set. Set it with:")
    print("   export OPENAI_API_KEY='your-api-key'")
else:
    print("✓ OPENAI_API_KEY is set")

## Part 1: Submitting Jobs

Create a workspace and submit some batch jobs.

In [ ]:
# Create a workspace - all jobs will be stored here
workspace_dir = "example_workspace"
relay = RelayClient(directory=workspace_dir)

print(f"✓ Created workspace: {workspace_dir}")

In [ ]:
# Create some batch requests
requests = [
    BatchRequest(
        id="req-1",
        model="gpt-4o-mini",
        system_prompt="You are a helpful assistant.",
        prompt="What is 2+2?",
        provider_args={}
    ),
    BatchRequest(
        id="req-2",
        model="gpt-4o-mini",
        system_prompt="You are a helpful assistant.",
        prompt="What is the capital of France?",
        provider_args={}
    ),
    BatchRequest(
        id="req-3",
        model="gpt-4o-mini",
        system_prompt="You are a helpful assistant.",
        prompt="Explain quantum computing in one sentence.",
        provider_args={}
    ),
]

print(f"Created {len(requests)} requests")

In [ ]:
# Submit the batch job with a unique ID
job_id = "notebook-demo-001"

try:
    job = relay.submit_batch(
        requests=requests,
        job_id=job_id,
        provider="openai",
        description="Notebook demonstration batch job"
    )
    
    print(f"✓ Job submitted successfully!")
    print(f"  Job ID: {job.job_id}")
    print(f"  Submitted at: {job.submitted_at}")
    print(f"  Status: {job.status}")
    print(f"  Number of requests: {job.n_requests}")
    
except ValueError as e:
    print(f"✗ Error: {e}")
    print("This might mean the job already exists. Try a different job_id.")

## Part 2: Tracking Jobs

List all jobs in the workspace and monitor their progress.

In [ ]:
# List all jobs in the workspace
all_jobs = relay.list_jobs()
print(f"Found {len(all_jobs)} job(s) in workspace:")
for jid in all_jobs:
    print(f"  - {jid}")

In [ ]:
# Get detailed information about a specific job
job_info = relay.get_job(job_id)
if job_info:
    print(f"Job: {job_info['job_id']}")
    print(f"  Description: {job_info['description']}")
    print(f"  Provider: {job_info['provider']}")
    print(f"  Status: {job_info['status']}")
    print(f"  Submitted: {job_info['submitted_at']}")
    print(f"  Requests: {job_info['n_requests']}")
    print(f"  Completed: {job_info.get('completed_requests', 0)}")
    print(f"  Failed: {job_info.get('failed_requests', 0)}")

In [ ]:
# Monitor job progress
print(f"Monitoring job: {job_id}")
print("=" * 60)

max_checks = 30  # Maximum number of status checks
check_interval = 5  # Check every 5 seconds

for i in range(max_checks):
    job_status = relay.monitor_batch(job_id)
    
    print(f"\n[{i * check_interval}s] Status: {job_status.status}")
    if hasattr(job_status, 'completed_requests'):
        print(f"  Progress: {job_status.completed_requests}/{job_status.n_requests} completed")
    
    if job_status.status == "completed":
        print("\n✓ Batch job completed!")
        break
    elif job_status.status == "failed":
        print("\n✗ Batch job failed!")
        break
    elif job_status.status in ["validating", "in_progress", "finalizing"]:
        print(f"  Waiting... (checking again in {check_interval}s)")
        time.sleep(check_interval)
    else:
        print(f"  Unknown status, waiting...")
        time.sleep(check_interval)
else:
    print(f"\n⚠ Maximum checks ({max_checks}) reached. Job may still be processing.")

## Part 3: Getting Results from Old Jobs

This is the key feature - you can create a new RelayClient instance with the same workspace directory and access all previous jobs and their results.

In [ ]:
# Simulate starting a new session - create a new RelayClient with the same workspace
print("Simulating a new session...")
print(f"Creating new RelayClient with workspace: {workspace_dir}")

new_relay = RelayClient(directory=workspace_dir)
print("✓ New RelayClient created")

# All jobs are still accessible!
existing_jobs = new_relay.list_jobs()
print(f"\nFound {len(existing_jobs)} existing job(s):")
for jid in existing_jobs:
    print(f"  - {jid}")

In [ ]:
# Check if results exist for a job
if new_relay.has_results(job_id):
    print(f"✓ Results exist for {job_id}")
else:
    print(f"✗ No results found for {job_id}")
    print("  (Results are saved when you call retrieve_batch_results)")

In [ ]:
# Retrieve results - this will fetch from API if not cached, or use cache if available
print(f"Retrieving results for {job_id}...")
print("=" * 60)

try:
    results = new_relay.retrieve_batch_results(job_id)
    
    print(f"\n✓ Retrieved {len(results)} results")
    print(f"\nResults are automatically saved to: {workspace_dir}/{job_id}_results.json")
    
    # Display sample results
    print("\n" + "=" * 60)
    print("Sample Results:")
    print("=" * 60)
    
    for i, result in enumerate(results[:3], 1):  # Show first 3
        print(f"\nResult {i}:")
        custom_id = result.get('custom_id', 'N/A')
        print(f"  Request ID: {custom_id}")
        
        # Extract response based on OpenAI format
        if 'response' in result:
            response = result['response']
            if 'body' in response:
                body = response['body']
                if 'output' in body:
                    output = body['output']
                    # Truncate long outputs
                    output_str = str(output)
                    if len(output_str) > 200:
                        output_str = output_str[:200] + "..."
                    print(f"  Output: {output_str}")
        
        if 'error' in result:
            print(f"  Error: {result['error']}")
    
    if len(results) > 3:
        print(f"\n  ... and {len(results) - 3} more results")
        
except ValueError as e:
    print(f"✗ Error: {e}")
except Exception as e:
    print(f"✗ Unexpected error: {e}")

In [ ]:
# Get results from cache (doesn't fetch from API)
print("Getting results from cache (no API call)...")
cached_results = new_relay.get_results(job_id)

if cached_results:
    print(f"✓ Found {len(cached_results)} cached results")
    print("  (These were loaded from disk, no API call was made)")
else:
    print("✗ No cached results found")
    print("  (Call retrieve_batch_results first to fetch and cache results)")

In [ ]:
# Force refresh results (bypass cache)
print("Force refreshing results (bypassing cache)...")
try:
    fresh_results = new_relay.retrieve_batch_results(job_id, force_refresh=True)
    print(f"✓ Retrieved {len(fresh_results)} fresh results from API")
except Exception as e:
    print(f"✗ Error: {e}")

## Summary

This notebook demonstrated:

1. **Submitting Jobs**: Create a workspace and submit batch jobs with unique IDs
2. **Tracking Jobs**: List jobs, get job info, and monitor progress
3. **Getting Results from Old Jobs**: Create a new RelayClient with the same workspace to access all previous jobs and results

### Key Benefits:

- **Persistent Storage**: All jobs and results are saved to the workspace directory
- **Session Independence**: You can close and reopen notebooks - all jobs are still accessible
- **Result Caching**: Results are cached on disk, so you don't need to re-fetch from the API
- **Easy Sharing**: Share the workspace directory to share all jobs and results

### Workspace Structure:

```
example_workspace/
  notebook-demo-001.json              # Job metadata
  notebook-demo-001_results.json       # Results (when retrieved)
  ...
```